# Скрипт для расчета параметров разрабатывваемого планера

Данный скрипт предназначен для расчета основных параметров модельных планеров для дальнейшего их проектирования в любой CAD среде. 

Примечание: данный скрипт написан в соответствии с пошаговой инструкцией "Step-by-step guide to calculating a light aircraft.pdf"

### ШАГ 1 - Техническое задание

Параметры на этом шаге задаются вами в соответствии с вашеми целями

In [172]:
#вес мотора
w_motor = 53 #грамм

#количество моторов
n_motor = 1 #шт

# вес регулятора скорости
w_ESC = 27 #грамм

# Количество регуляторов скорости
n_ESC = 1 #шт

# Примечание: Количество моторов может отличаться от количества регуляторов скорости:
# n_motor >= n_ESC
# Связано это с тем, что к некоторым моделям ESC можно подключить более одного двигателя

# вес приемника управления
w_receiver = 15 #грамм, модель "FS-iA6B"
# Примечание: на борту планера могут быть и другие приёмники/передатчики. Основной приемник всегда один.

#вес сервопривода
w_servo = 13.4 #грамм, модель "TowerPro MG90"

#количество сервоприводов
n_servo = 4 #шт: на 2 Элерона, 1 Киль и 1 стабилизатор

#вес аккумуляторной ячейки
w_bat_cell = 49 #грамм, аккумуляторы типоразмера 18650

#количество ячеек в сборке
n_bat_cell = 3 #шт: схема сборки 3S1P

#вес платы BMS
w_BMS = 8.7 #грамм, модель "https://market.yandex.ru/product--bms-kontroller-3s-40a-dlia-li-ion-iacheek-3-7v-18650-26650/962042505?cpc=2T9kpZW_tO7nKULPmvl-cva4O6LDJVEop73t09BR7d3y2ij9fpFed_mt24qkdfwQJ7Pf54fCx7iH058HGpijKiwhtUS_xHXpyrsLpwIxzeCVOFnJ8aZybHKGxXlB3MZ5vFFjLrIPIIUv9uOFnuHc82L9xa8Sz6aJhyRD8PvePM2yJCqBYsF0pUAJXoRPGebJCdcoqBKZ5Y1Pk8BY0jZ_cD9ljHs6ZXBMkdryP_U8z2kQvmxP5Sf6SoNRSHBEGSYnNzJr1iWyUS-KdgHCFAMtlQ%2C%2C&from-show-uid=16834608512406912852400001&sku=101324535512&do-waremd5=veTjfe9vPLUWdF41vjtbng&sponsored=1&cpa=1"

#вес аккумуляторной сборки/покупного аккумулятора
w_bat = (w_bat_cell*n_bat_cell + w_BMS)*1.1 # вес самостоятельно собираемой батареи
# w_bat = 145 # вес покупного аккумулятора

#вес полезной нагрузки
w_load = 250 #грамм, под полезной нагрузкой можно понимать любое доп. оборудование

# Скорость сваливания в штопор
V_pilling = 50 # км/ч



## Вес и геометрические размеры

### ШАГ 2 - Определение взлетной массы в первом приближении

In [173]:
# Расчет взлетного веса планера в первом приближении
G_insides = (w_motor*n_motor + w_ESC*n_ESC + w_servo*n_servo + w_bat + w_receiver + w_load)
G_insides

569.87

In [174]:
G_plane = G_insides*2
G_plane

1139.74

### ШАГ 3 - Определение массы элементов конструкции

In [175]:

# Функция по уточненному расчету массы планера через долю его элементов от общей массы
def mass_correction(w_plane,w_delta):
    w_wing = (w_plane-w_delta)*0.17
    w_fuselage = (w_plane-w_delta)*0.15
    w_shassis = (w_plane-w_delta)*0.07
    w_stabs = (w_plane-w_delta)*0.04
    w_controlsys = (w_plane-w_delta)*0.02
    w_plane_corr = w_plane - G_insides- w_wing -  w_fuselage - w_shassis - w_stabs - w_controlsys
    prst_delta = w_plane_corr/w_plane*100
    return prst_delta ,w_plane-w_plane_corr ,w_plane, w_wing , w_fuselage, w_shassis, w_stabs, w_controlsys


In [176]:
calc = mass_correction (G_plane,0)
calc    # Вывод рассчитанных масс:
        # Скорректированная масса самолета, гр
        # Первоначальная масса самолета, гр
        # Масса крыла, гр
        # Масса шасси, гр
        # Масса стабилизатора, гр
        # Масса органов управления, гр       

(4.999999999999999,
 1082.753,
 1139.74,
 193.75580000000002,
 170.96099999999998,
 79.7818,
 45.589600000000004,
 22.794800000000002)

In [177]:
G_plane = calc[1]
G_plane

1082.753

На данном этапе мы получили скорректированную массу планера, а именно снизили её на 5%. 

Примечание: в исходном документе при расчете уточненной массы на ШАГЕ 3 допущена ошибка: складывается число "62" вместо "72", в результате чего получается 4 ШАГ - уточнение результатов. 4 Шага быть впринципе не должно, потому что какие бы мы числа не подставили, разница между грубой прикидкой и более точным расчетом всегда составит 5%. В связи с этим в нашем расчете ШАГ 4  мы пропускаем и сразу переходим к шагу 5.

### ШАГ 5 - Определение потребной площади крыла

Зная взлетный вес самолета и требуемую скорость сваливания определяем потребную площадь крыла по
формуле:

$$S_{кр}= 207 \frac{G_{взл}}{C_{ya_{max}}* V_{св}^2}$$

In [178]:
C_ya_max = 1.78 #Профиль P-III (15,5%), угол атаки 24 градуса
S_wing_min = 207*G_plane/1000/C_ya_max/(V_pilling^2)
S_wing_min

2.623242872191011